In [1]:
import pyomo.environ as pyo
from pyomo.environ import *
from pyomo.opt import SolverFactory
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import plotly.io as pio
from math import pi
import sympy as sp


In [2]:
# Dataset
# Marginal Emission Intensity of the Grid
Main_EI =  np.loadtxt('Main_EI.csv', delimiter=',')

# Electricity Retail Price $/kWh
Retail_Price =  np.loadtxt('HOEP.csv', delimiter=',')


In [ ]:
# Simulation Data

# Time horizon
Times = 4340

#per unit value
V_base = 12.66 #kV
S_base = 100 # kVA
I_base = S_base / (V_base * np.sqrt(3)) # A
Z_base = 1000 * (V_base ** 2) / S_base # Ohm

# Carbon price
Carbon_Price = 80 # 80 $/tCO2

# ESS capacity
ES_Capacity = 40 # 4 MWh

# ESS charging/discharging efficiency and C-rate
Eta = 0.97
Crate = 0.25

In [4]:
#DN OPF
class DNOPF:
    def __init__(self):
        self.model = pyo.ConcreteModel()
        self.build_model()

    def build_model(self):

        DN = self.model        
        DN.times = pyo.RangeSet(0, Times - 1) # Set for time intevals
        DN.Main_P = pyo.Var(DN.times, within=Reals) # Net power exchange with the grid, Purchase +, Sell - 
        DN.ES_ch = pyo.Var(DN.times, within=NonNegativeReals) # Charging +
        DN.ES_dis = pyo.Var(DN.times, within=NonNegativeReals)
        
        # Objective functions (7)
        def objective(DN):
            return sum(( Retail_Price[t] + Carbon_Price * Main_EI[t]) * DN.Main_P[t] for t in DN.times) 
        DN.Objective = pyo.Objective(rule=objective, sense=pyo.minimize)

        # ESS Power balance (6)
        def DNBalanceP(DN, t):
            return DN.Main_P[t] - (DN.ES_ch[t] - DN.ES_dis[t]) == 0
        DN.Constraint1 = Constraint(DN.times, rule=DNBalanceP) 

        # Net power exchange upper limit
        def MainUpperLimitPB(DN, t):
            return DN.Main_P[t]<= 1e3
        DN.Constraint2 = pyo.Constraint(DN.times, rule=MainUpperLimitPB) 

        # Net power exchange lower limit
        def MainUpperLimitPS(DN, t):
            return DN.Main_P[t] >= -1e3
        DN.Constraint3 = pyo.Constraint(DN.times, rule=MainUpperLimitPS) 

        # ESS charging power upper limit
        def ESUpperLimitch(DN, t):
            return DN.ES_ch[t] <= Crate * ES_Capacity
        DN.Constraint4 = pyo.Constraint(DN.times, rule=ESUpperLimitch)
        
        # ESS charging power lower limit
        def ESLowerLimitch(DN, t):
            return DN.ES_ch[t] >= 0
        DN.Constraint5 = pyo.Constraint(DN.times, rule=ESLowerLimitch)

        # ESS discharging power upper limit
        def ESUpperLimitdis(DN, t):
            return DN.ES_dis[t] <= Crate * ES_Capacity
        DN.Constraint6 = pyo.Constraint(DN.times, rule=ESUpperLimitdis)
        
        # ESS discharging power lower limit
        def ESLowerLimitdis(DN, t):
            return DN.ES_dis[t] >= 0
        DN.Constraint7 = pyo.Constraint(DN.times, rule=ESLowerLimitdis)

        # ESS SOC upper limit
        def ESSOCUpperLimit(DN, t):
            return sum(Eta * DN.ES_ch[s] - DN.ES_dis[s]/Eta for s in range (0,t+1))  <= ES_Capacity
        DN.Constraint8 = pyo.Constraint(DN.times, rule=ESSOCUpperLimit)

        # ESS SOC lower limit
        def ESSOCLowerLimit(DN, t):
            return sum(Eta * DN.ES_ch[s] - DN.ES_dis[s]/Eta for s in range (0,t+1)) >= 0
        DN.Constraint9 = pyo.Constraint(DN.times, rule=ESSOCLowerLimit)

    def solve(self):
        DN = self.model
        self.ES_SOC = np.zeros((Times))
        opt = pyo.SolverFactory("gurobi")
        result = opt.solve(DN, tee=True)

        # Primal_Variables
        self.Main_P = np.array([DN.Main_P[t].value for t in DN.times])
        self.ES_ch = np.array([DN.ES_ch[t].value for t in DN.times])
        self.ES_dis = np.array([DN.ES_dis[t].value for t in DN.times])

        self.DN_Obj = DN.Objective()

        # ESS SOC
        for t in DN.times:
            self.ES_SOC[t] = sum(self.ES_ch[s] - self.ES_dis[s] for s in range (0,t+1))/ES_Capacity

        return result


In [5]:
DN_OPF = DNOPF()
DN_OPF.solve()

# Primal_Variables
A_Main_P = DN_OPF.Main_P
A_ES_ch = DN_OPF.ES_ch
A_ES_dis = DN_OPF.ES_dis
A_ES_SOC = DN_OPF.ES_SOC

# Performance Results
R_ESS = sum((Retail_Price[t] + Carbon_Price * Main_EI[t] ) * A_Main_P[t] for t in range (Times)) # ESS Revenue
E_ESS = sum( Main_EI[t] * (A_Main_P[t]) for t in range(Times)) # ESS Carbon Emissions
ESS_Life= sum( A_ES_ch[t] + A_ES_dis[t] for t in range(Times))/ES_Capacity # ESS Lifetime

Read LP format model from file C:\Users\ASUS\AppData\Local\Temp\tmppfasgp8o.pyomo.lp
Reading time = 0.17 seconds
x1: 3906 rows, 1302 columns, 381486 nonzeros
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11+.0 (26100.2))

CPU model: 13th Gen Intel(R) Core(TM) i9-13900H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 3906 rows, 1302 columns and 381486 nonzeros
Model fingerprint: 0x4e44e03a
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+01, 3e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 1e+03]
Presolve removed 3473 rows and 435 columns
Presolve time: 0.06s
Presolved: 433 rows, 1297 columns, 188785 nonzeros

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 9.353e+04
 Factor NZ  : 9.396e+04 (roughly 1 MB of memory)
 Factor Ops : 

In [6]:
# Plot Fig. 4

# K = 20 # iteration count for ESS capacity
# E = 20 # iteration count for carbon price
# ES_Capacity = 0
# Carbon_Price = 0

# KE = np.zeros((K,E)) # Overall emission reduction at each iteration k and e

# for k in range (K):

#     ES_Capacity = 0.1 + 4 * k 

#     for e in range (E):
#         Carbon_Price = 0.08 + e * 0.006
        
#         DN_OPF = DNOPF()
#         DN_OPF.solve()

#         # Primal_Variables
#         A_Main_P = DN_OPF.Main_P
#         A_ES_ch = DN_OPF.ES_ch
#         A_ES_dis = DN_OPF.ES_dis
#         A_ES_SOC = DN_OPF.ES_SOC

#         KE[k,e] = sum( Main_EI[t] * A_Main_P[t] for t in range(Times))
